In [ ]:
pip install elasticsearch==8.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 385 kB 4.0 MB/s 
     |████████████████████████████████| 59 kB 3.7 MB/s 
     |████████████████████████████████| 140 kB 66.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [ ]:
pip install pyspellchecker==0.5.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.2 MB/s 


In [ ]:
import pandas as pd
import openpyxl
import random
import math
import collections.abc
import copy
import ast
import json
from google.colab import files
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import spellchecker
from spellchecker import SpellChecker
import numpy

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
!git clone "<github_json_data_repo_link>"

Cloning into 'IR_Project'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 128 (delta 105), reused 106 (delta 97), pack-reused 0
Receiving objects: 100% (128/128), 15.85 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (105/105), done.
Checking out files: 100% (107/107), done.


In [ ]:
%cd /content/IR_Project
!git pull

/content/IR_Project
Already up to date.


In [ ]:
calendar_df=pd.read_excel("/content/IR_Project/dimdate-QueryResult.xlsx")

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
calendar_df=calendar_df.drop(['word_date','date','quarter','yearmonth','week','month_and_year','timezone_offset','timezone','timezone_id'],axis=1)

print(calendar_df)

      sasdate  date_key  year  month  day_of_month day_of_week  weekday  \
0       14610     10100  2000    1.0           1.0         Sat      7.0   
1       14611     10200  2000    1.0           2.0         Sun      1.0   
2       14612     10300  2000    1.0           3.0         Mon      2.0   
3       14613     10400  2000    1.0           4.0         Tue      3.0   
4       14614     10500  2000    1.0           5.0         Wed      4.0   
...       ...       ...   ...    ...           ...         ...      ...   
9386    23996     91225  2025    9.0          12.0         Fri      6.0   
9387    23997     91325  2025    9.0          13.0         Sat      7.0   
9388    23998     91425  2025    9.0          14.0         Sun      1.0   
9389    23999     91525  2025    9.0          15.0         Mon      2.0   
9390    24000     91625  2025    9.0          16.0         Tue      3.0   

      holiday  
0     NEWYEAR  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
...

In [ ]:
calendar_index_generated_events=pd.DataFrame(columns=['datekey','sasdate','year','month','day','weekday_num','event'])
date_list=pd.DataFrame(columns=['datekey','sasdate','year','month','day','weekday_num'])
for row in calendar_df.iterrows():
    if(isinstance(row[1]['holiday'],str)):
        new_row=[row[1]['date_key'],row[1]['sasdate'],row[1]['year'],row[1]['month'],row[1]['day_of_month'],row[1]['weekday'],str(row[1]['holiday'].lower().split())]
        calendar_index_generated_events.loc[len(calendar_index_generated_events)]=new_row

    date_list.loc[len(date_list)]=[row[1]['date_key'],row[1]['sasdate'],row[1]['year'],row[1]['month'],row[1]['day_of_month'],row[1]['weekday']]
print(calendar_index_generated_events)

    datekey sasdate  year  month   day  weekday_num               event
0     10100   14610  2000    1.0   1.0          7.0         ['newyear']
1     11700   14626  2000    1.0  17.0          2.0             ['mlk']
2     21400   14654  2000    2.0  14.0          2.0      ['valentines']
3     22100   14661  2000    2.0  21.0          2.0    ['uspresidents']
4     42300   14723  2000    4.0  23.0          1.0          ['easter']
..      ...     ...   ...    ...   ...          ...                 ...
469   52625   23887  2025    5.0  26.0          2.0        ['memorial']
470   61525   23907  2025    6.0  15.0          1.0         ['fathers']
471   70125   23923  2025    7.0   1.0          3.0          ['canada']
472   70425   23926  2025    7.0   4.0          6.0  ['usindependence']
473   90125   23985  2025    9.0   1.0          2.0           ['labor']

[474 rows x 7 columns]


In [ ]:
calendar_index_generated_events=calendar_index_generated_events.set_index(['sasdate'])

In [ ]:
calendar_index_generated_events.index[1]


14626

### used a slang language normalization list from https://www.kaggle.com/datasets/841dab43e3e9047809fa046ad6fe0e3f97879c9eba46f44752829b01897ed93c?datasetId=525901&select=Test_Set_3802_Pairs.txt

In [ ]:
slang_dictionary=dict()
with open("/content/IR_Project/emnlp_dict.txt","r") as slang_file:
  for each_line in slang_file:
    #print(each_line.strip().split("\t"))
    each_line_split=each_line.strip().split("\t")
    slang_dictionary[each_line_split[0]]=each_line_split[1]

In [ ]:
N=len(calendar_index_generated_events)
word=""
for i in range(0,N):
  #print(read_json_df.loc[read_json_df['user']==row[0][0]])
  event_str=calendar_index_generated_events.loc[(calendar_index_generated_events.index[i]),'event']
  formatted_event=""
  #print(event_str)
  for character in event_str:
    if character.isalnum():
      formatted_event=formatted_event+character.lower()
      word=word+character.lower()
    if character==' ' or character==',':
      if word in slang_dictionary:
        #print(i,read_json_df.index[i][0],read_json_df.index[i][1], word)
        formatted_event=formatted_event.replace(word,slang_dictionary[word])
      word=""
      formatted_event=formatted_event+character
    if character==']':
      if word in slang_dictionary:
        #print(i,read_json_df.index[i][0],read_json_df.index[i][1], word)
        formatted_event=formatted_event.replace(word,slang_dictionary[word])
      word=""

  formatteed_event=formatted_event.split(',')
  calendar_index_generated_events.loc[(calendar_index_generated_events.index[i]),'formatted_event']=formatted_event
  #print(i)


In [ ]:
calendar_index_generated_events

,datekey,year,month,day,weekday_num,event,formatted_event
sasdate,,,,,,,
14610,10100,2000,1.0,1.0,7.0,['newyear'],newyear
14626,11700,2000,1.0,17.0,2.0,['mlk'],mlk
14654,21400,2000,2.0,14.0,2.0,['valentines'],valentines
14661,22100,2000,2.0,21.0,2.0,['uspresidents'],uspresidents
14723,42300,2000,4.0,23.0,1.0,['easter'],easter
...,...,...,...,...,...,...,...
23887,52625,2025,5.0,26.0,2.0,['memorial'],memorial
23907,61525,2025,6.0,15.0,1.0,['fathers'],fathers
23923,70125,2025,7.0,1.0,3.0,['canada'],canada


In [ ]:
#contains all the generated calendars
calendar_index_date_key=pd.DataFrame(columns=['user','datekey','sasdate','year','month','day','weekday_num','event'])


In [ ]:
json_file=open('/content/IR_Project/JSON/user_date_indexed/user_event_1000.json')
dict_json=json.load(json_file)
read_json_df=pd.DataFrame(dict_json['data'])

In [ ]:
read_json_df=read_json_df.set_index(['user','sasdate'])

In [ ]:
read_json_df

datekey    year  month   day  weekday_num  \
user      sasdate                                               
user_1    22497.0   80521.0  2021.0    8.0   5.0          5.0   
          16109.0   20804.0  2004.0    2.0   8.0          1.0   
          17968.0   31209.0  2009.0    3.0  12.0          5.0   
          17692.0   60908.0  2008.0    6.0   9.0          2.0   
          22805.0   60922.0  2022.0    6.0   9.0          5.0   
...                     ...     ...    ...   ...          ...   
user_1000 22890.0   90222.0  2022.0    9.0   2.0          6.0   
          18897.0   92711.0  2011.0    9.0  27.0          3.0   
          16496.0   30105.0  2005.0    3.0   1.0          3.0   
          22596.0  111221.0  2021.0   11.0  12.0          6.0   
          17531.0  123107.0  2007.0   12.0  31.0          2.0   

                                      event  
user      sasdate                            
user_1    22497.0             ['tim event']  
          16109.0   ['Joey birthday party']  
          17968.0         ['rhea Birthday']  
          17692.0      ['meet mom and dad']  
          22805.0         ['johny Meeting']  
...                                     ...  
user_1000 22890.0  ['birthday party mummy']  
          18897.0      ['bday mom and dad']  
          16496.0     ['anniversary mummy']  
          22596.0          ['ShivaJayanti']  
          17531.0                  ['holi']  

[11994 rows x 6 columns]

In [ ]:
N=len(read_json_df)
word=""
for i in range(0,N):
  #print(read_json_df.loc[read_json_df['user']==row[0][0]])
  event_str=read_json_df.loc[(read_json_df.index[i][0],read_json_df.index[i][1]),'event']
  formatted_event=""
  #print(event_str)
  for character in event_str:
    if character.isalnum():
      formatted_event=formatted_event+character.lower()
      word=word+character.lower()
    if character==' ' or character==',':
      if word in slang_dictionary:
        #print(i,read_json_df.index[i][0],read_json_df.index[i][1], word)
        formatted_event=formatted_event.replace(word,slang_dictionary[word])
      word=""
      formatted_event=formatted_event+character
    if character==']':
      if word in slang_dictionary:
        #print(i,read_json_df.index[i][0],read_json_df.index[i][1], word)
        formatted_event=formatted_event.replace(word,slang_dictionary[word])
      word=""

  formatteed_event=formatted_event.split(',')
  read_json_df.loc[(read_json_df.index[i][0],read_json_df.index[i][1]),'formatted_event']=formatted_event
  #print(i)


In [ ]:
read_json_df

datekey    year  month   day  weekday_num  \
user      sasdate                                               
user_1    22497.0   80521.0  2021.0    8.0   5.0          5.0   
          16109.0   20804.0  2004.0    2.0   8.0          1.0   
          17968.0   31209.0  2009.0    3.0  12.0          5.0   
          17692.0   60908.0  2008.0    6.0   9.0          2.0   
          22805.0   60922.0  2022.0    6.0   9.0          5.0   
...                     ...     ...    ...   ...          ...   
user_1000 22890.0   90222.0  2022.0    9.0   2.0          6.0   
          18897.0   92711.0  2011.0    9.0  27.0          3.0   
          16496.0   30105.0  2005.0    3.0   1.0          3.0   
          22596.0  111221.0  2021.0   11.0  12.0          6.0   
          17531.0  123107.0  2007.0   12.0  31.0          2.0   

                                      event       formatted_event  
user      sasdate                                                  
user_1    22497.0             ['tim event']             tim event  
          16109.0   ['Joey birthday party']   joey birthday party  
          17968.0         ['rhea Birthday']         rhea birthday  
          17692.0      ['meet mom and dad']      meet mom and dad  
          22805.0         ['johny Meeting']        johnny meeting  
...                                     ...                   ...  
user_1000 22890.0  ['birthday party mummy']  birthday party mummy  
          18897.0      ['bday mom and dad']  birthday mom and dad  
          16496.0     ['anniversary mummy']     anniversary mummy  
          22596.0          ['ShivaJayanti']          shivajayanti  
          17531.0                  ['holi']                  holi  

[11994 rows x 7 columns]

In [ ]:
for index,data in read_json_df.iterrows():
  print(index[0]+'_'+str(int(index[1])))
  break

user_1_22497


In [ ]:
es_user='elastic'
es_password="FnkDbmxTmqG2vpkvdSuNfhi6"
cloud_id_val="IR_deployment:dXMtd2VzdDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ5YmE1ZmUxMTA1NGU0NzNhODRjMDdjZWM4ZWFlYjZlNiQ4NWQ5NmIyYzJlYzA0YTZiYTI3OWFkZDZmZTI0Y2YwMw=="
es = Elasticsearch(cloud_id=cloud_id_val,basic_auth=(es_user,es_password))


In [ ]:
def cals_as_docs(read_json_df,index_name):
  for index,data in read_json_df.iterrows():
    yield{
        "_index":index_name,
        "_id":index[0]+'_'+str(int(index[1])),
        "_source":{'user':index[0],'sasdate':index[1],'datekey':data['datekey'],'year':data['year'],'month':data['month'],'day':data['day'],'weekday_num':data['weekday_num'],'event':data['formatted_event']}
    }



In [ ]:
def festivals_as_docs(read_json_df,index_name):
  for index,data in read_json_df.iterrows():
    yield{
        "_index":index_name,
        "_id":index,
        "_source":{'sasdate':index,'datekey':data['datekey'],'year':data['year'],'month':data['month'],'day':data['day'],'weekday_num':data['weekday_num'],'event':data['formatted_event']}
    }

In [ ]:
helpers.bulk(es,cals_as_docs(read_json_df,'index_username_date_new'))

(11994, [])

In [ ]:
helpers.bulk(es,festivals_as_docs(calendar_index_generated_events,'festivals_index'))

(474, [])

IR elastic search deployed here: https://cloud.elastic.co/deployments/d50ee153cb7d457eafe10d70101aafed
Call API: https://ir-deployment.kb.us-west1.gcp.cloud.es.io:9243/app/dev_tools#/console

> GET index_username_date/_search
> {
>   "query":{
>    "wildcard":{
>      "user":{
>        "value":"user_1*"       
>      }
>    }
>  }
>}






In [ ]:
json_file=open('/content/IR_Project/JSON/generated_events_sasdate.json')
dict_json=json.load(json_file)
read_json_df=pd.DataFrame(dict_json['data'])

In [ ]:
read_json_df=read_json_df.set_index(['event','sasdate'])

In [ ]:
N=len(read_json_df)
word=""
for i in range(0,N):
  #print(read_json_df.loc[read_json_df['user']==row[0][0]])
  event_str=read_json_df.index[i][0]
  formatted_event=""
  #print(event_str)
  for character in event_str:
    if character.isalnum():
      formatted_event=formatted_event+character.lower()
      word=word+character.lower()
    if character==' ' or character==',':
      if word in slang_dictionary:
        #print(i,read_json_df.index[i][0],read_json_df.index[i][1], word)
        formatted_event=formatted_event.replace(word,slang_dictionary[word])
      word=""
      formatted_event=formatted_event+character
    if character==']':
      if word in slang_dictionary:
        #print(i,read_json_df.index[i][0],read_json_df.index[i][1], word)
        formatted_event=formatted_event.replace(word,slang_dictionary[word])
      word=""

  formatteed_event=formatted_event.split(',')
  read_json_df.loc[(read_json_df.index[i][0],read_json_df.index[i][1]),'formatted_event']=formatted_event
  #print(i)
  
json_of_read_json_df = read_json_df.to_json(orient="table")
json_load=json.loads(json_of_read_json_df)
json_dump=json.dumps(json_load,indent=4)
file_name='expanded_slang_events_generated_events.json'
with open(file_name,'w') as event_file:
  event_file.write(json_dump)
#files.download(file_name)


In [ ]:
read_json_df=read_json_df.reset_index()
read_json_df=read_json_df.set_index(['formatted_event','sasdate'])

In [ ]:
read_json_df

In [ ]:
json_of_read_json_df = read_json_df.to_json(orient="table")
json_load=json.loads(json_of_read_json_df)
json_dump=json.dumps(json_load,indent=4)
file_name='expanded_slang_events_generated_events.json'
with open(file_name,'w') as event_file:
  event_file.write(json_dump)
#files.download(file_name)